In [1]:
"""Données JSON et programmation objet"""
################################################################################
# fichier  : exo2.py
# Auteur : RAKOTOSON Loic
################################################################################

In [2]:
################################################################################
# Importation de fonctions externes :
import json
import pandas as pd
from requests import get, HTTPError
# Definition locale de fonctions :

In [3]:
def getJsonData(base, dico):
    with get(base, params = dico) as response:
        return response.json()

In [4]:
################################################################################
# Definition des classes :

In [5]:
class Residence():
    
    def __init__(self, dico):
        self.nom = dico['nom']
        self.desc = dico['description']
        self.lat = dico['latitude']
        self.lon = dico['longitude']
        self.zone = dico['zone']
    
    def __repr__(self):
        chaine = "Résidence {} ({}): {}".format(
            self.nom.upper(),
            self.zone,
            self.desc
        )
        return chaine
    
    def distanceDe(self, longitude, latitude, profil):

        base_url = "https://api.openrouteservice.org/matrix"
        source = ",".join([self.lon, self.lat])
        dest = ",".join([longitude, latitude])
        params = {
            "api_key" : APIKEY,
            "profile": profil,
            "locations": "|".join([source, dest]),
            "sources": "0",
            "destinations": "1",
            "metrics": "distance",
            "units": "m",
        }

        data = getJsonData(base_url, params)
        return data['distances'][0][0]
    
    def dureeDe(self, longitude, latitude, profil):

        base_url = "https://api.openrouteservice.org/matrix"
        source = ",".join([self.lon, self.lat])
        dest = ",".join([longitude, latitude])
        params = {
            "api_key" : APIKEY,
            "profile": profil,
            "locations": "|".join([source, dest]),
            "sources": "0",
            "destinations": "1",
            "metrics": "duration"
        }

        data = getJsonData(base_url, params)
        return data['durations'][0][0]

In [6]:
APIKEY = '5b3ce3597851110001cf62486cfda2a850de46e4a43253fa6adffd31'

# Exercice 2: Données JSON et programmation objet
Le but de l'exercice est de créer et d'utiliser des objets «Résidence» représentant les résidences du CROUS.

## Question 1
à l'aide du  module `json` de  Python charger les  données du  fichier `residences.json` dans    une   variable jsonData

In [7]:
with open("residences.json", "r", encoding="UTF-8") as json_file:
    jsonData = json.load(json_file)

## Question 2
créer une classe `Residence` et  son   constructeur qui,   à partir d'un   dictionnaire contenant les  données d'une résidence, crée   un  objet `Residence`

In [8]:
r1 = Residence(jsonData[0])

## Question 3
ajouter à la  classe `Residence` une   méthode spéciale d'affichage pour    qu'une résidence s'affiche avec    un  `print()`

In [9]:
print(r1)

Résidence LA GARE (Rennes Centre): Résidence du centre ville de Rennes à proximité immédiate de la gare SNCF, de la gare routière et du métro.


## Question 4.1
ajouter à la  classe `Residence` une   `méthodedistanceDe()` qui   prend comme premiers paramètres deux    coordonnées GPS  (une    longitude et  une   latitude, toutes deux    sous    forme d'une chaîne de  caractères), en  troisième paramètre un  profil de  parcours (par   exemple 'driving-car') et  qui   renvoie la  distance en  mètres entre la  résidence et  ces   coordonnées, selon le  profil de  parcours spécifié.

In [10]:
UnivR2 = ("-1.702147", "48.118737")
profil = "driving-car"
r1.distanceDe(*UnivR2, profil)

4038.98

## Question 4.2
sur   le  même principe, ajouter à la  classe `Residence` une   `méthodedureeDe()` qui   renvoie la  durée de  parcours en  secondes entre la  résidence et  des   coordonnées GPS  fournies en  paramètres, selon un  profil de  parcours spécifié lui  aussi    en  paramètre.

In [11]:
r1.dureeDe(*UnivR2, profil)

575.52

## Question 5
dans    le  programme principal, écrire le  code    Python permettant d'afficher les  résidences de  Rennes se  trouvant à **moins de  20  minutes à pied** du  campus de  Villejean avec    la  durée correspondante en  minutes

In [12]:
list_residence = [Residence(res) for res in jsonData]
profil = "foot-walking"

In [13]:
for residence in list_residence:
    if "Rennes" in residence.zone:
        duree = residence.dureeDe(*UnivR2, profil)/60
        if  duree <= 20:
            print(
                "A {} minutes: {}".format(
                    int(duree),
                    str(residence)
                )
            )

A 14 minutes: Résidence VILLEJEAN OUEST (Rennes ouest-Villejean):  Résidence proche de l'université de Rennes II et campus santé
A 15 minutes: Résidence LA HARPE (Rennes ouest-Villejean): Résidence proche du campus de Villejean. Restaurant universitaire A  proximité.
A 9 minutes: Résidence LANGUEDOC (Rennes ouest-Villejean): Résidence proche du campus de Villejean.
A 16 minutes: Résidence BARBARA (Rennes ouest-Villejean): Résidence proche du campus de Villejean. 
A 17 minutes: Résidence BEAUREGARD (Rennes ouest-Villejean): Résidence proche du campus de Villejean.
A 5 minutes: Résidence ALSACE (Rennes ouest-Villejean): En face du campus de Rennes 2, à proximité du RU et du campus santé.
A 14 minutes: Résidence LA TOUCHE (Rennes ouest-Villejean): Résidence de studios proche du Campus de Villejean, du Centre Ville et du METRO Anatole France .


## Question 6
dans    le  programme principal, écrire le  code    Python permettant d'afficher les  résidences en  dehors de  Rennes (l'attribut zone ne  contient pas   'Rennes'),  et  la  **distance en  voiture** entre la  résidence et  le  campus de  Villejean, en  kilomètres.

In [14]:
profil = "driving-car"

for residence in list_residence:
    if "Rennes" not in residence.zone:
        distance = residence.distanceDe(*UnivR2, profil)/1000
        print(
            "A {} km: {}".format(
                round(distance, 2),
                str(residence)
            )
        )

A 239.46 km: Résidence KERGOAT (Brest): A proximité des UFR Droit/Sciences Economiques, AES, STAPS et de l'IUT. 
A 238.76 km: Résidence LE BOUGUEN (Brest): Campus de l'UBO (Sciences, Lettres, IUFM, Médecine). Restaurant universitaire sur place. 
A 238.64 km: Résidence LANREDEC (Brest): Campus de l'UBO. Restaurant universitaire à proximité.
A 210.38 km: Résidence QUIMPER RUE DE L'UNIVERSITE (Quimper):  Chambres près de l'IUT. Restaurant universitaire et caféteria sur place. 
A 115.35 km: Résidence LANN TRUSSAC (Vannes): Près de l'IUT et du Lycée Lesage. Restaurant université à proximité. 
A 152.01 km: Résidence LANVEUR (Lorient): Proche du campus universitaire. Face au restaurant universitaire . Maison des étudiants. 
A 163.64 km: Résidence EDOUARD BRANLY (Lannion): Chambres rénovées avec sanitaires individuels. A coté de l'IUT. Restaurant universitaire et cafétéria sur place.
A 95.33 km: Résidence GERNUGAN (Saint Brieuc): Près du pôle universitaire. Bus A, B, C1, C2
A 95.28 km: Résiden